In [2]:
import requests
import time
import pandas as pd
import re   
from bs4 import BeautifulSoup

In [ ]:
# %pip install cloudscraper

import cloudscraper

# Use existing notebook variables if present to avoid unnecessary re-creation.

if "url" not in globals():
	url = "https://www.wheree.com/get-brands?categories%5B%5D=703&page=1&category_id=701&category_slug=Local_Services&location_id=231&location_level=0&location_slug=United_States"

# reuse existing scraper if available and valid, else create a new one
if "scraper" not in globals():
	scraper = cloudscraper.create_scraper()  # automatically handles Cloudflare challenge
else:
	try:
		from cloudscraper import CloudScraper
		if not isinstance(scraper, CloudScraper):
			scraper = cloudscraper.create_scraper()
	except Exception:
		scraper = cloudscraper.create_scraper()

# retry loop to avoid ReadTimeout errors
max_retries = 5
timeout = 60  # increase timeout
backoff = 2

resp = None
for attempt in range(1, max_retries + 1):
	try:
		resp = scraper.get(url, timeout=timeout)
		resp.raise_for_status()
		break
	except Exception as e:
		# handle read timeout and other transient errors with backoff
		print(f"Attempt {attempt} failed: {e}")
		if attempt < max_retries:
			time.sleep(backoff * attempt)
		else:
			print("Max retries reached, giving up.")

if resp is None:
	raise RuntimeError(f"Failed to fetch {url} after {max_retries} attempts.")

print("status:", resp.status_code)
print("preview:", resp.text[:1000])

# try to parse JSON using existing helper if available
if "_safe_json_parse" in globals():
	payload = _safe_json_parse(resp)
	if payload is not None:
		print("parsed JSON keys (sample):", list(payload.keys())[:20] if isinstance(payload, dict) else type(payload))
else:
	try:
		payload = resp.json()
		print("parsed JSON keys (sample):", list(payload.keys())[:20] if isinstance(payload, dict) else type(payload))
	except Exception:
		print("response is not valid JSON")

status: 200
preview: {"mapRows":{"current_page":1,"data":[{"id":6255982,"name":"Mr. Appliance Of Milwaukee","fulladdress":"S83W18745 Saturn Dr, Muskego, WI 53150, United States","alias":"mr-appliance-of-milwaukee","ranking":2147483647,"rating":4.9,"number_of_rates":0,"price_level":0,"created_at":"2025-10-01T19:10:45.000000Z","longitude":-88.148119,"latitude":42.892868,"city_id":12689,"category_id":705,"category2_id":703,"category3_id":711,"short_description":"","image":"https:\/\/static.where-e.com\/United_States\/Racine\/Mr-Appliance-Of-Milwaukee_6d7066365d5bc77cc10614fddf4f4088.jpg","openTime":[{"day_in_week":4,"time_open":28800,"time_close":61200}],"sub_cat":[{"id":705,"name":"Appliances & Repair Services","slug":"Appliances_Repair","parent_id":701,"attribute_value":0,"desc":null,"created_at":"2024-02-15T04:50:58.000000Z","updated_at":"2024-04-05T01:54:25.000000Z","on_menu":0,"order":0,"icon":null,"image":"\/2024\/04\/05\/4f84c51f9975bee6242a317305d8623416949f15.jpg","content":null,

In [4]:
# scrape_all_pages_laundromarts.py
# Reuse existing notebook imports and helpers (scraper, extract_maprows_info, pd, time, re, max_retries, timeout)

# helper: normalize mapRows / brands HTML into a list of records
def extract_maprows_info(parsed):
    """
    Accepts parsed response (dict from .json() or raw HTML string) and returns
    a list of normalized record dicts. This is tolerant to several payload shapes:
      - dict with 'mapRows' -> {'data': [...]}
      - dict with 'brands' containing HTML
      - raw HTML containing brand blocks with data-brand attributes
    Returns [] on failure.
    """
    records = []
    try:
        # case: JSON dict with mapRows.data as list
        if isinstance(parsed, dict):
            mr = parsed.get("mapRows")
            if isinstance(mr, dict):
                data = mr.get("data")
                if isinstance(data, list) and data:
                    return data
            # sometimes there's a top-level 'data' list
            if isinstance(parsed.get("data"), list):
                return parsed.get("data")
            # try HTML inside 'brands' key
            brands_html = parsed.get("brands")
            if isinstance(brands_html, str):
                parsed = brands_html  # fall through to HTML parsing

        # case: parsed is HTML string -> use BeautifulSoup to extract blocks
        if isinstance(parsed, str):
            soup = BeautifulSoup(parsed, "html.parser")
            # common container class used on site: category__top__item (fallback to any element with data-brand)
            items = soup.find_all("div", class_=re.compile(r"category__top__item|category__item|category-item"))
            if not items:
                items = soup.find_all(attrs={"data-brand": True})
            for div in items:
                rec = {}
                rec["id"] = div.get("data-brand") or div.get("data-id") or None
                a = div.find("a")
                if a:
                    rec["url"] = a.get("href")
                    # try to get a title element if present
                    title = a.find(["h1", "h2", "h3", "h4", "h5"])
                    if title:
                        rec["name"] = title.get_text(strip=True)
                    else:
                        # fallback to anchor/text content
                        rec["name"] = a.get_text(separator=" ", strip=True)
                else:
                    rec["url"] = None
                    rec["name"] = div.get_text(separator=" ", strip=True)
                # optional image
                img = div.find("img")
                if img:
                    rec["image"] = img.get("src") or img.get("data-src")
                    if not rec.get("name"):
                        rec["name"] = img.get("alt")
                # include raw snippet for diagnostics
                rec["raw_html"] = str(div)
                records.append(rec)
            return records
    except Exception as e:
        # keep function safe for use in loops
        print("extract_maprows_info parse error:", e)
    return records


# ensure scraper is available
if "scraper" not in globals():
    try:
        scraper = cloudscraper.create_scraper()
    except Exception:
        raise RuntimeError("cloudscraper not available to create scraper")

# determine base URL (use existing `url` if present)
if "url" in globals() and isinstance(url, str):
    base_url = url
else:
    base_url = "https://www.wheree.com/get-brands?categories%5B%5D=703&page=1&category_id=701&category_slug=Local_Services&location_id=231&location_level=0&location_slug=United_States"

# pages to fetch: 1..908
pages = list(range(1,909))

# retry/backoff settings (reuse if defined)
_retries = globals().get("max_retries", 5)
_timeout = globals().get("timeout", 60)
_backoff = 2

all_records = []

for p in pages:
    # build page URL by replacing page=... or appending if missing
    if re.search(r"page=\d+", base_url):
        page_url = re.sub(r"page=\d+", f"page={p}", base_url)
    else:
        sep = "&" if "?" in base_url else "?"
        page_url = f"{base_url}{sep}page={p}"

    resp = None
    for attempt in range(1, _retries + 1):
        try:
            resp = scraper.get(page_url, timeout=_timeout)
            resp.raise_for_status()
            break
        except Exception as e:
            print(f"[page {p}] attempt {attempt} failed: {e}")
            if attempt < _retries:
                time.sleep(_backoff * attempt)
            else:
                print(f"[page {p}] max retries reached, skipping page.")
    if resp is None:
        continue

    # try to parse JSON; prefer resp.json() but fall back to text for extract_maprows_info
    parsed = None
    try:
        parsed = resp.json()
    except Exception:
        parsed = resp.text

    # use existing extract_maprows_info (defined in notebook) to normalize records
    try:
        recs = extract_maprows_info(parsed)
    except Exception as e:
        print(f"[page {p}] extract_maprows_info failed: {e}")
        recs = []

    if not recs:
        print(f"[page {p}] no records extracted")
    else:
        all_records.extend(recs)
        print(f"[page {p}] extracted {len(recs)} records (total so far: {len(all_records)})")

# build dataframe and save
if not all_records:
    print("No records collected from pages 1..20.")
else:
    df_all = pd.DataFrame(all_records)
    out_fname = "laundromats.csv"
    df_all.to_csv(out_fname, index=False)
    display(df_all.head(20))
    print(f"Saved {len(df_all)} records to {out_fname}") 
# Reuse existing notebook imports and helpers (scraper, extract_maprows_info, pd, time, re, max_retries, timeout)



[page 1] extracted 10 records (total so far: 10)
[page 2] extracted 10 records (total so far: 20)
[page 3] extracted 10 records (total so far: 30)
[page 4] extracted 10 records (total so far: 40)
[page 5] extracted 10 records (total so far: 50)
[page 6] extracted 10 records (total so far: 60)
[page 7] extracted 10 records (total so far: 70)
[page 8] extracted 10 records (total so far: 80)
[page 9] extracted 10 records (total so far: 90)
[page 10] extracted 10 records (total so far: 100)
[page 11] extracted 10 records (total so far: 110)
[page 12] extracted 10 records (total so far: 120)
[page 13] extracted 10 records (total so far: 130)
[page 14] extracted 10 records (total so far: 140)
[page 15] extracted 10 records (total so far: 150)
[page 16] extracted 10 records (total so far: 160)
[page 17] extracted 10 records (total so far: 170)
[page 18] extracted 10 records (total so far: 180)
[page 19] extracted 10 records (total so far: 190)
[page 20] extracted 10 records (total so far: 200

,id,name,fulladdress,alias,ranking,rating,number_of_rates,price_level,created_at,longitude,...,city_id,category_id,category2_id,category3_id,short_description,image,openTime,sub_cat,affiliate,city
0,6255982,Mr. Appliance Of Milwaukee,"S83W18745 Saturn Dr, Muskego, WI 53150, United...",mr-appliance-of-milwaukee,2147483647,4.9,0,0,2025-10-02T02:10:45.000000Z,-88.148119,...,12689,705,703,711,,https://static.where-e.com/United_States/Racin...,"[{'day_in_week': 4, 'time_open': 28800, 'time_...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 12689, 'timezone': None}"
1,6242673,Sherman's Warehouse Clearance Center,"1203 E Marietta Ave, Peoria Heights, IL 61616",sherman-s-warehouse-clearance-center,2147483647,4.8,0,0,2025-09-27T01:00:23.000000Z,-89.577461,...,13228,735,713,703,,https://static.where-e.com/United_States/Peori...,"[{'day_in_week': 4, 'time_open': 36000, 'time_...","[{'id': 735, 'name': 'Appliances Stores', 'slu...",None,"{'id': 13228, 'timezone': None}"
2,3584218,Brandts Appliance Parts & Services,"4415 77th St, Vero Beach, FL 32967",brandts-appliance-parts-services-1,2147483647,4.9,0,0,2025-06-06T21:17:00.000000Z,-80.430491,...,12236,705,703,706,,https://static.where-e.com/United_States/Brand...,"[{'day_in_week': 4, 'time_open': 28800, 'time_...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 12236, 'timezone': None}"
3,3583514,Convenient Appliance Service,"4018 Country Club Rd, Winston-Salem, NC 27104,...",convenient-appliance-service,2147483647,4.9,0,0,2025-06-06T21:14:11.000000Z,-80.318848,...,12761,705,703,706,,https://static.where-e.com/United_States/Conve...,"[{'day_in_week': 4, 'time_open': 28800, 'time_...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 12761, 'timezone': None}"
4,3742616,Clean Rite Center,"4618 A Edmondson Ave, Baltimore, MD 21229, Uni...",clean-rite-center-13,2048544665,4.9,0,0,2025-06-08T02:14:27.000000Z,-76.694793,...,12317,703,706,705,,https://static.where-e.com/United_States/Clean...,"[{'day_in_week': 4, 'time_open': 60, 'time_clo...","[{'id': 703, 'name': 'Laundromats', 'slug': 'L...",None,"{'id': 12317, 'timezone': None}"
5,5851544,Appliances USA Pro,"2429 Americana Blvd, Orlando, FL 32839",appliances-usa-pro-1,2027375284,4.9,0,0,2025-09-07T03:50:30.000000Z,-81.414712,...,13016,705,703,718,,https://static.where-e.com/United_States/Orang...,"[{'day_in_week': 4, 'time_open': 60, 'time_clo...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 13016, 'timezone': None}"
6,5761785,Mr. Appliance of Tampa Bay,"600 S Magnolia Ave Suite 190, Tampa, FL 33606,...",mr-appliance-of-tampa-bay-1,1905821775,4.8,0,0,2025-09-06T00:26:34.000000Z,-82.465495,...,12666,705,703,706,,https://static.where-e.com/United_States/Hills...,"[{'day_in_week': 4, 'time_open': 60, 'time_clo...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 12666, 'timezone': None}"
7,3777182,"All American Appliance Service,LLC.","7584 Olive Blvd #207, St. Louis, MO 63130, Uni...",all-american-appliance-service-llc,1847683736,4.7,0,0,2025-06-08T19:21:15.000000Z,-90.335353,...,12565,705,703,706,,https://static.where-e.com/United_States/All-A...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 12565, 'timezone': None}"
8,3583134,Mr. Appliance of Cincinnati,"11144 Luschek Dr, Cincinnati, OH 45241, United...",mr-appliance-of-cincinnati,1834133660,4.8,0,0,2025-06-06T21:12:47.000000Z,-84.379893,...,10830,705,703,706,,https://static.where-e.com/United_States/Mr-Ap...,"[{'day_in_week': 4, 'time_open': 60, 'time_clo...","[{'id': 705, 'name': 'Appliances & Repair Serv...",None,"{'id': 10830, 'timezone': None}"
9,3681921,Washland Laundry Inc,"116 N 19th Ave, Melrose Park, IL 60160, United...",washland-laundry-inc,1786370029,4.9,0,0,2025-06-07T14:19:25.000000Z,-87.856970,...,12209,703,706,705,,https://static.where-e.com/United_States/Washl...,"[{'day_in_week': 4, 'time_open': 21600, 'time_...","[{'i

Saved 9080 records to laundromats.csv


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("laundromats.csv")
df.head()

,id,name,fulladdress,alias,ranking,rating,number_of_rates,price_level,created_at,longitude,...,city_id,category_id,category2_id,category3_id,short_description,image,openTime,sub_cat,affiliate,city
0,3766375,-,"2455 Whipple Ave NW, Canton, OH 44708, United ...",brand-ab950709,31160754,4.3,0,0,2025-06-08T05:57:59.000000Z,-81.425068,...,14015,706,703,702,NaN,https://static.where-e.com/United_States/_ec06...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 14015, 'timezone': None}"
1,3847843,$2.79 Cleaners,"100 Lake Ave, Maitland, FL 32751, United States",2-79-cleaners,55630976,4.4,0,0,2025-06-13T12:40:39.000000Z,-81.365724,...,12421,706,702,703,NaN,https://static.where-e.com/United_States/2-79-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12421, 'timezone': None}"
2,3721639,$3.00 Pink Cleaners,"1240 Middle Country Rd, Selden, NY 11784, Unit...",3-00-pink-cleaners,22261966,4.1,0,0,2025-06-07T22:12:40.000000Z,-73.056162,...,11274,706,702,703,NaN,https://static.where-e.com/United_States/3-00-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 11274, 'timezone': None}"
3,3720588,$3.99 A Cleaners,"87 Boston Post Rd, Orange, CT 06477, United St...",3-99-a-cleaners,71624324,4.7,0,0,2025-06-07T22:01:43.000000Z,-72.989665,...,12096,706,702,703,NaN,https://static.where-e.com/United_States/3-99-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12096, 'timezone': None}"
4,1990718,1 Clean Laundry,"4036 13th St, St Cloud, FL 34769, United States",1-clean-laundry,0,0.0,0,0,2025-02-19T19:44:21.000000Z,-81.311374,...,6141,706,703,735,NaN,https://static.where-e.com/United_States/1-Cle...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 6141, 'timezone': None}"


In [3]:
import re

In [4]:



# Load your dataset (adjust filename if needed)
# df = pd.read_csv('your_file.csv')

# Example pattern: captures two-letter state abbreviation before ZIP code
state_pattern = re.compile(r'\b([A-Z]{2})\b(?=\s*\d{5})')

def extract_state(address):
    """Extracts state abbreviation (e.g., FL, NY, CA) from address string."""
    if pd.isna(address):
        return None
    match = state_pattern.search(address)
    if match:
        return match.group(1)
    return None

# Apply function to your actual column
df['state'] = df['fulladdress'].apply(extract_state)

# Preview results
print(df.head())


        id                 name  \
0  3766375                    -   
1  3847843       $2.79 Cleaners   
2  3721639  $3.00 Pink Cleaners   
3  3720588     $3.99 A Cleaners   
4  1990718      1 Clean Laundry   

                                         fulladdress               alias  \
0  2455 Whipple Ave NW, Canton, OH 44708, United ...      brand-ab950709   
1    100 Lake Ave, Maitland, FL 32751, United States       2-79-cleaners   
2  1240 Middle Country Rd, Selden, NY 11784, Unit...  3-00-pink-cleaners   
3  87 Boston Post Rd, Orange, CT 06477, United St...     3-99-a-cleaners   
4    4036 13th St, St Cloud, FL 34769, United States     1-clean-laundry   

    ranking  rating  number_of_rates  price_level  \
0  31160754     4.3                0            0   
1  55630976     4.4                0            0   
2  22261966     4.1                0            0   
3  71624324     4.7                0            0   
4         0     0.0                0            0   

             

In [5]:
df.head()

,id,name,fulladdress,alias,ranking,rating,number_of_rates,price_level,created_at,longitude,...,category_id,category2_id,category3_id,short_description,image,openTime,sub_cat,affiliate,city,state
0,3766375,-,"2455 Whipple Ave NW, Canton, OH 44708, United ...",brand-ab950709,31160754,4.3,0,0,2025-06-08T05:57:59.000000Z,-81.425068,...,706,703,702,NaN,https://static.where-e.com/United_States/_ec06...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 14015, 'timezone': None}",OH
1,3847843,$2.79 Cleaners,"100 Lake Ave, Maitland, FL 32751, United States",2-79-cleaners,55630976,4.4,0,0,2025-06-13T12:40:39.000000Z,-81.365724,...,706,702,703,NaN,https://static.where-e.com/United_States/2-79-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12421, 'timezone': None}",FL
2,3721639,$3.00 Pink Cleaners,"1240 Middle Country Rd, Selden, NY 11784, Unit...",3-00-pink-cleaners,22261966,4.1,0,0,2025-06-07T22:12:40.000000Z,-73.056162,...,706,702,703,NaN,https://static.where-e.com/United_States/3-00-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 11274, 'timezone': None}",NY
3,3720588,$3.99 A Cleaners,"87 Boston Post Rd, Orange, CT 06477, United St...",3-99-a-cleaners,71624324,4.7,0,0,2025-06-07T22:01:43.000000Z,-72.989665,...,706,702,703,NaN,https://static.where-e.com/United_States/3-99-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12096, 'timezone': None}",CT
4,1990718,1 Clean Laundry,"4036 13th St, St Cloud, FL 34769, United States",1-clean-laundry,0,0.0,0,0,2025-02-19T19:44:21.000000Z,-81.311374,...,706,703,735,NaN,https://static.where-e.com/United_States/1-Cle...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 6141, 'timezone': None}",FL


In [12]:
# Overwrite the same file with the updated data
df.to_csv("laundromats.csv", index=False)
print("✅ Changes appended (file updated successfully).")

✅ Changes appended (file updated successfully).


In [7]:

import ast  # safely evaluate string representation of list/dict


# Function to extract id and name
def extract_ids_and_names(detail_str):
    try:
        data_list = ast.literal_eval(detail_str)  # safely parse string to Python list
        return [(item['id'], item['name']) for item in data_list]
    except Exception as e:
        return None

# Apply the function to create new columns
df['extracted'] = df['sub_cat'].apply(extract_ids_and_names)

# Optional: separate into two new columns for clarity
df['ids'] = df['extracted'].apply(lambda x: [i[0] for i in x] if x else None)
df['names'] = df['extracted'].apply(lambda x: [i[1] for i in x] if x else None)

print(df[['ids', 'names']])


                  ids                                              names
0     [706, 703, 702]  [Dry Cleaning Services, Laundromats, Sewing & ...
1     [706, 702, 703]  [Dry Cleaning Services, Sewing & Alterations S...
2     [706, 702, 703]  [Dry Cleaning Services, Sewing & Alterations S...
3     [706, 702, 703]  [Dry Cleaning Services, Sewing & Alterations S...
4     [706, 703, 735]  [Dry Cleaning Services, Laundromats, Appliance...
...               ...                                                ...
9075  [706, 702, 703]  [Dry Cleaning Services, Sewing & Alterations S...
9076  [706, 703, 735]  [Dry Cleaning Services, Laundromats, Appliance...
9077  [703, 705, 706]  [Laundromats, Appliances & Repair Services, Dr...
9078  [735, 706, 703]  [Appliances Stores, Dry Cleaning Services, Lau...
9079  [703, 706, 735]  [Laundromats, Dry Cleaning Services, Appliance...

[9080 rows x 2 columns]


In [10]:

# Function to extract the first name
def extract_first_name(detail_str):
    try:
        data_list = ast.literal_eval(detail_str)  # convert string to list of dicts
        if isinstance(data_list, list) and len(data_list) > 0:
            return data_list[0].get('name')
    except Exception:
        return None
    return None

# Apply function
df['category'] = df['sub_cat'].apply(extract_first_name)

print(df[['sub_cat', 'category']])


                                                sub_cat               category
0     [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
1     [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
2     [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
3     [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
4     [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
...                                                 ...                    ...
9075  [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
9076  [{'id': 706, 'name': 'Dry Cleaning Services', ...  Dry Cleaning Services
9077  [{'id': 703, 'name': 'Laundromats', 'slug': 'L...            Laundromats
9078  [{'id': 735, 'name': 'Appliances Stores', 'slu...      Appliances Stores
9079  [{'id': 703, 'name': 'Laundromats', 'slug': 'L...            Laundromats

[9080 rows x 2 columns]


In [11]:
df.head()

,id,name,fulladdress,alias,ranking,rating,number_of_rates,price_level,created_at,longitude,...,image,openTime,sub_cat,affiliate,city,state,extracted,ids,names,category
0,3766375,-,"2455 Whipple Ave NW, Canton, OH 44708, United ...",brand-ab950709,31160754,4.3,0,0,2025-06-08T05:57:59.000000Z,-81.425068,...,https://static.where-e.com/United_States/_ec06...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 14015, 'timezone': None}",OH,"[(706, Dry Cleaning Services), (703, Laundroma...","[706, 703, 702]","[Dry Cleaning Services, Laundromats, Sewing & ...",Dry Cleaning Services
1,3847843,$2.79 Cleaners,"100 Lake Ave, Maitland, FL 32751, United States",2-79-cleaners,55630976,4.4,0,0,2025-06-13T12:40:39.000000Z,-81.365724,...,https://static.where-e.com/United_States/2-79-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12421, 'timezone': None}",FL,"[(706, Dry Cleaning Services), (702, Sewing & ...","[706, 702, 703]","[Dry Cleaning Services, Sewing & Alterations S...",Dry Cleaning Services
2,3721639,$3.00 Pink Cleaners,"1240 Middle Country Rd, Selden, NY 11784, Unit...",3-00-pink-cleaners,22261966,4.1,0,0,2025-06-07T22:12:40.000000Z,-73.056162,...,https://static.where-e.com/United_States/3-00-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 11274, 'timezone': None}",NY,"[(706, Dry Cleaning Services), (702, Sewing & ...","[706, 702, 703]","[Dry Cleaning Services, Sewing & Alterations S...",Dry Cleaning Services
3,3720588,$3.99 A Cleaners,"87 Boston Post Rd, Orange, CT 06477, United St...",3-99-a-cleaners,71624324,4.7,0,0,2025-06-07T22:01:43.000000Z,-72.989665,...,https://static.where-e.com/United_States/3-99-...,"[{'day_in_week': 4, 'time_open': 25200, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 12096, 'timezone': None}",CT,"[(706, Dry Cleaning Services), (702, Sewing & ...","[706, 702, 703]","[Dry Cleaning Services, Sewing & Alterations S...",Dry Cleaning Services
4,1990718,1 Clean Laundry,"4036 13th St, St Cloud, FL 34769, United States",1-clean-laundry,0,0.0,0,0,2025-02-19T19:44:21.000000Z,-81.311374,...,https://static.where-e.com/United_States/1-Cle...,"[{'day_in_week': 4, 'time_open': 27000, 'time_...","[{'id': 706, 'name': 'Dry Cleaning Services', ...",NaN,"{'id': 6141, 'timezone': None}",FL,"[(706, Dry Cleaning Services), (703, Laundroma...","[706, 703, 735]","[Dry Cleaning Services, Laundromats, Appliance...",Dry Cleaning Services
